# Photometric redshifts for CFIS in DEEP2+3 field
Run random forest.

In [ ]:
import os
import sys

In [ ]:
py_v = '3.7'
path = f"{os.environ['CONDA_PREFIX']}/../shapepipe/lib/python{py_v}/site-packages/shapepipe-0.0.2-py{py_v}.egg"
sys.path.append(path)
from shapepipe.utilities.canfar import download, dir_list
from shapepipe.utilities.file_system import mkdir

In [ ]:
import params_ps3pi_cfis as params

In [ ]:
for dir_name in [params.catalog_path, params.matched_path, params.unmatched_path]:
    mkdir(dir_name)

## Pre-processing

### Clean-up previous run(s)

In [ ]:
! python photoz.py --survey ps3pi_cfis --clean True --input params_ps3pi_cfis

## Download input catalogs

In [ ]:
from io import StringIO
from contextlib import redirect_stdout
from shapepipe.utilities.canfar import download, dir_list, vosHandler

def Ddir_list(path, verbose=False):
    """list

    List content of path on vos

    Parameters
    ----------
    path : string
        path on vos, starts with 'vos:cfis/...'
    verbose : bool, optional, default=False
        verbose output if True

    Raises
    ------
    HTTPError, KeyError

    Returns
    -------
    vls_out : array of string
        file or directory at path
    """

    cmd = 'vls'
    sys.argv = [cmd, path]
    vls = vosHandler(cmd)

    if verbose:
        print('Getting vos directory content from vls...')

    f = StringIO()

    try:
        with redirect_stdout(f):
            vls()
    except:
        print('Error during vls command')
        raise

    vls_out = f.getvalue()
    return vls_out.split('\n')

In [ ]:
vos_dir = 'vos:cfis/cosmostat/catalogues/multi_band_matched/W3_deep23'

# ...
cat_name = 'R_CFHT_vs_CFIS.csv'
cat_path = f'{params.catalog_path}/{cat_name}'
download(f'{vos_dir}/{cat_name}', cat_path, verbose=True)

# Matched and unmatched catalogues
vos_dir_matched = f'{vos_dir}/ps3pi_cfis/matched'
vos_dir_unmatched = f'{vos_dir}/ps3pi_cfis/unmatched'

# If dir_list not found, replace with Ddir_list
f_list = dir_list(vos_dir_matched)
for f in f_list:
    download(f'{vos_dir_matched}/{f}', f'{params.matched_path}/{f}', verbose=True)
f_list = dir_list(vos_dir_unmatched)
for f in f_list:
    download(f'{vos_dir_unmatched}/{f}', f'{params.unmatched_path}/{f}', verbose=True)

### Create invidivual tile catalogs

In [ ]:
! python photoz.py --survey ps3pi_cfis --make True --input params_ps3pi_cfis

### Join individual tile catalogs

In [ ]:
! python photoz.py --survey ps3pi_cfis --join True --input params_ps3pi_cfis

### Create plots of the redshift distributions
Figures are saves as `pdf` in `output/ps3pi_cfis/CFIS_matched_deep_2_3_catalog_R/figures`.

In [ ]:
! python photoz.py --survey ps3pi_cfis --generate_plots True --input params_ps3pi_cfis

## Machine-learning

### Train random forest

In [ ]:
! python photoz.py --survey ps3pi_cfis --learning True --preprocess drop --algorithm RF --optimize HyperOpt --input params_ps3pi_cfis